In [1]:
from cqlengine import columns
from cqlengine.models import Model
from cqlengine import connection
from cqlengine.management import sync_table
CASSANDRA_KEYSPACE = "playground"
connection.setup(['172.31.39.226'], CASSANDRA_KEYSPACE)
import json

In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
#df = sqlContext.read.json("s3n://patricks3db/reviews_Movies_and_TV_small.json")
df = sqlContext.read.json("s3n://patricks3db/reviews_Movies_and_TV.json")
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [3]:
ratings = df
ratings = ratings.drop("helpful")
ratings = ratings.drop("reviewText")
ratings = ratings.drop("reviewTime")
ratings = ratings.drop("reviewerName")
ratings = ratings.drop("summary")
ratings = ratings.drop("unixReviewTime")
ratings.columns

['asin', 'overall', 'reviewerID']

In [4]:
#users = ratings.map(lambda r: r.reviewerID).distinct().zipWithIndex()
#users.take(5)
#users.count()

In [5]:
#movies = ratings.map(lambda r: r.asin).distinct().zipWithIndex()
#movies.take(5)
#movies.count()

In [6]:
#moviesDf = sqlContext.createDataFrame(movies, ['asin', 'movieID'])
#moviesDf.take(5)
#usersDf = sqlContext.createDataFrame(users, ['reviewerID', 'userID'])
#usersDf.take(5)

In [7]:
#rawRatings1 = ratings.join(usersDf, ratings.reviewerID == usersDf.reviewerID).drop("reviewerID")

In [8]:
#rawRatings2 = rawRatings1.join(moviesDf, ratings.asin == moviesDf.asin).drop("asin")

In [9]:
#rawRatings3 = rawRatings2.map(lambda row: (row.userID, row.movieID, row.overall)).cache()

In [10]:
intmax = 2 ** 31 - 1
rawRatings3 = ratings.map(lambda row: (abs(hash(row.reviewerID))%intmax, abs(hash(row.asin))%intmax, row.overall)).cache()

In [11]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [12]:
rank = 10
numIterations = 10
model = ALS.train(rawRatings3, rank, numIterations)

In [13]:
#model.recommendProducts(8751,5)

In [14]:
#model.userFeatures().take(5)

In [15]:
#userMoviePair = usersDf.join(moviesDf).drop('reviewerID').drop('asin')
#userMoviePair.printSchema()
#userMoviePair = userMoviePair.filter(userMoviePair.userID < 1000)
#userMoviePair = userMoviePair.map(lambda row: (row.userID, row.movieID)).cache()

In [16]:
moviePool = sqlContext.read.json("s3n://patricks3db/meta_Movies_and_TV.json")
#moviePool.printSchema()
print moviePool.count()
moviePool = moviePool.filter(moviePool.imUrl.endswith('jpg') | moviePool.imUrl.endswith('jpeg'))
print moviePool.count()
intmax = 2 ** 31 - 1
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
createIntegerId = udf(lambda text: abs(hash(text))%intmax, IntegerType())
moviePool = moviePool.withColumn('pid', createIntegerId(moviePool.asin))
#moviePool.printSchema()
moviePool = moviePool.select('pid', 'asin', 'brand', 'imUrl', 'price', 'title')
moviePool.first()

208321
197474


Row(pid=485290944, asin=u'0000589012', brand=None, imUrl=u'http://ecx.images-amazon.com/images/I/519%2B1kseM3L._SY300_.jpg', price=15.95, title=u"Why Don't They Just Quit? DVD Roundtable Discussion: What Families and Friends need to Know About Addiction and Recovery")

In [17]:
def AddToCassandra_allcountsbatch_bypartition(d_iter):
        class movieCatalog2(Model):
                from cqlengine import columns
                from cqlengine.models import Model
                from cqlengine import connection
                from cqlengine.management import sync_table
                CASSANDRA_KEYSPACE = "playground"
                pid = columns.Integer(primary_key=True)
                asin = columns.Text(primary_key=True)
                brand = columns.Text()
                imUrl = columns.Text()
                price = columns.Float()
                title = columns.Text()
        connection.setup(['172.31.39.226'], CASSANDRA_KEYSPACE)
        sync_table(movieCatalog2)
        for d in d_iter:
                movieCatalog2.create(**d)
# Create table if it does not exist. Need to do this before submitting to Spark to avoid collisions
AddToCassandra_allcountsbatch_bypartition([])

In [18]:
moviePool.toJSON(use_unicode=False).map(lambda row: json.loads(row)).foreachPartition(AddToCassandra_allcountsbatch_bypartition)

In [19]:
userPool = df.select('reviewerID', 'reviewerName').distinct()
intmax = 2 ** 31 - 1
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
createIntegerId = udf(lambda text: abs(hash(text))%intmax, IntegerType())
userPool = userPool.withColumn('uid', createIntegerId(userPool.reviewerID))
userPool.printSchema()

root
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- uid: integer (nullable = true)



In [20]:
def AddToCassandra_allcountsbatch_bypartition(d_iter):
        class userbase2(Model):
                from cqlengine import columns
                from cqlengine.models import Model
                from cqlengine import connection
                from cqlengine.management import sync_table
                CASSANDRA_KEYSPACE = "playground"
                uid = columns.Integer(primary_key=True)
                reviewerID = columns.Text(primary_key=True)
                reviewerName = columns.Text()
        connection.setup(['172.31.39.226'], CASSANDRA_KEYSPACE)
        sync_table(userbase2)
        for d in d_iter:
                userbase2.create(**d)
# Create table if it does not exist. Need to do this before submitting to Spark to avoid collisions
AddToCassandra_allcountsbatch_bypartition([])

In [21]:
userPool.toJSON(use_unicode=False).map(lambda row: json.loads(row)).foreachPartition(AddToCassandra_allcountsbatch_bypartition)

In [22]:
userPool.count()

2098332

In [45]:
#userMoviePair = userPool.select('uid').join(moviePool.select('pid')).cache()
#userMoviePair.printSchema()
#userMoviePair.rdd.getNumPartitions()

In [46]:
#recommendations = model.predictAll(userMoviePair.rdd)
#print 'Patrick: done!'

In [47]:
#recommendationsToC = recommendations.map(lambda x: {"user":x[0], "product":x[1], "rating":x[2]}).cache()

In [48]:
def AddToCassandra_allcountsbatch_bypartition(d_iter):
        from cqlengine import columns
        from cqlengine.models import Model
        from cqlengine import connection
        from cqlengine.management import sync_table
        CASSANDRA_KEYSPACE = "playground"
        class predictions3(Model):
                user = columns.Integer(primary_key=True)
                product = columns.Integer()
                rating = columns.Float(primary_key=True, clustering_order="DESC")
        connection.setup(['172.31.39.226'], CASSANDRA_KEYSPACE)
        sync_table(predictions3)
        for d in d_iter:
                predictions3.create(**d)

# Create table if it does not exist. Need to do this before submitting to Spark to avoid collisions
AddToCassandra_allcountsbatch_bypartition([])
#recommendationsToC.foreachPartition(AddToCassandra_allcountsbatch_bypartition)


In [56]:
testuid = 1981581556
recommendations = model.recommendProducts(testuid, 20000)
rmap = {'uid':testuid, 'rmap':dict([(x[1], x[2]) for x in recommendations])}

In [91]:
#compute recommmendation and save to cassandra
def createRecommendations(uid, uname):
    from pyspark.sql.functions import udf
    from pyspark.sql.types import IntegerType
    #addUid = udf(lambda inputUid: testuid, IntegerType())
    #userMoviePair = moviePool.select('pid')
    #print userMoviePair.printSchema()
    #print type(userMoviePair)
    #userMoviePair2 = userMoviePair.withColumn('uid', addUid(userMoviePair.pid)).select('uid','pid')
    #print userMoviePair2.printSchema()
    #print userMoviePair2.take(5)
    recommendations = model.recommendProducts(testuid, 20000)
    return {'user':uid, 'userName':uname, 'rmap':dict([(x[1], x[2]) for x in recommendations])}
    #recommendationsToC = recommendations.map(lambda x: {"user":x[0], "product":x[1], "rating":x[2]}).cache()
# createRecommendations(testuid, 'Patrick')

In [87]:
def loadRecommendations(d_iter):
        from cqlengine import columns
        from cqlengine.models import Model
        from cqlengine import connection
        from cqlengine.management import sync_table
        CASSANDRA_KEYSPACE = "playground"
        class predictions4(Model):
                user = columns.Integer(primary_key=True)
                rmap = columns.Map(columns.Integer(), columns.Float())
        connection.setup(['172.31.39.226'], CASSANDRA_KEYSPACE)
        sync_table(predictions4)
        for d in d_iter:
                predictions4.create(**d)

# Create table if it does not exist. Need to do this before submitting to Spark to avoid collisions
loadRecommendations([])
#recommendationsToC.foreachPartition(AddToCassandra_allcountsbatch_bypartition)

In [81]:
userPool.printSchema()
#result = userPool.map(lambda x: createRecommendations(x.uid, x.reviewerName))
uids = userPool.map(lambda x:x.uid).collect()
uids[0]
#result.foreachPartition(loadRecommendations)

root
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- uid: integer (nullable = true)

446818794


In [89]:
from cassandra.cluster import Cluster
cluster = Cluster(['172.31.39.223', '172.31.39.224'])
session = cluster.connect()
import json

In [92]:
for uid in uids:
    recommendations = model.recommendProducts(uid, 2000)
    jsonObject = {'user':uid, 'rmap':dict([(x[1], x[2]) for x in recommendations])}
    session.execute("insert into playground.predictions4 json '" + json.dumps(jsonObject) + "';")

Traceback (most recent call last):
  File "/usr/lib/python2.7/SocketServer.py", line 295, in _handle_request_noblock
ERROR:py4j.java_gateway:Error while sending or receiving.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.8.2.1-src.zip/py4j/java_gateway.py", line 479, in send_command
    raise Py4JError("Answer from Java side is empty")
Py4JError: Answer from Java side is empty
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.8.2.1-src.zip/py4j/java_gateway.py", line 425, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib/python2.7/socket.py", line 224, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 39381)
----------------------------------------


    self.process_request(request, client_address)
  File "/usr/lib/python2.7/SocketServer.py", line 321, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python2.7/SocketServer.py", line 334, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python2.7/SocketServer.py", line 649, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pyspark/serializers.py", line 544, in read_int
    raise EOFError
EOFError


Py4JNetworkError: An error occurred while trying to connect to the Java server

In [ ]:
"""
userPool.printSchema()
#userList = userPool.map(lambda row: row.uid).take(50)
userList = userPool.map(lambda row: row.uid).collect()
#print userList
"""

In [ ]:
#for uid in userList: createRecommendationsForAll(uid)

In [ ]:
"""
testuid = 1981581556
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
addUid = udf(lambda inputUid: testuid, IntegerType())
userMoviePair = moviePool.select('pid')
#print userMoviePair.printSchema()
#print type(userMoviePair)
userMoviePair2 = userMoviePair.withColumn('uid', addUid(userMoviePair.pid)).select('uid','pid')
#print userMoviePair2.printSchema()
#print userMoviePair2.take(5)
recommendations = model.predictAll(userMoviePair2.rdd.cache())
print recommendations.take(5)
recommendationsToC = recommendations.map(lambda x: {"user":x[0], "product":x[1], "rating":x[2]}).cache()
print recommendationsToC.take(5)
recommendationsToC.foreachPartition(AddToCassandra_allcountsbatch_bypartition)
"""